<a href="https://colab.research.google.com/github/tanveer08/Developed-a-Parkinson-s-Disease-detection-model-using-audio-spectrograms/blob/main/MIMO_OTFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

M = 4
N = 4

T = 1
ALPHA_WIDTH = 10
n_samples_per_T = 100
delta_t = T / n_samples_per_T
t = np.arange(-ALPHA_WIDTH * 10, ALPHA_WIDTH * 10 + delta_t, delta_t)

In [ ]:
def alpha_kl(k, l, T, t):
    s = np.zeros(t.shape)
    for n in range(N):
        current_signal = np.exp(1j * 2 * np.pi * k * n / N) * M / T * np.exp(1j * np.pi * M * (t - l / M * T - n * T)) * np.sinc(M / T * (t - l * T / M - n * T))
        s = s + current_signal
    return np.sqrt(T / M / N) * s

alpha_signals = np.zeros((M, N, t.shape[0]), dtype='complex128')
for k in range(N):
    for l in range(M):
        s = alpha_kl(k, l, T, t)
        alpha_signals[k, l] = s

In [ ]:
#Data from Tx-1

data_tx_1 = np.zeros((M, N), dtype='complex128')
data_tx_1[0, 0] = 1.0
#Data from Tx-2
data_tx_2 = np.zeros((M, N), dtype='complex128')
data_tx_2[0, 0] = 1.0
#Clubbing the data togther
#data_transmitted= np.zeros(((M, N),2), dtype='complex128')
X = []
X.append(data_tx_1)
X.append(data_tx_2)
print(X)
print(data_tx_1[0, 0])
print(X[1][0,0])

[array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]]), array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])]
(1+0j)
(1+0j)


In [ ]:
data_signal_1 = np.zeros(t.shape, dtype='complex128')
data_signal_2 = np.zeros(t.shape, dtype='complex128')
for k in range(N):
    for l in range(M):
        data_signal_1 += data_tx_1[k, l] * alpha_kl(k, l, T, t)
        data_signal_2 += data_tx_2[k, l] * alpha_kl(k, l, T, t)


Channel Paths Coefficients for each Tx-Rx  Pair

In [ ]:
h_vals_11 = np.array([1.0, 0.4 + 0.3j, -0.2 + 0.1j])
tau_vals_int_11 = [0, 1, 2] # to be scaled by T / N
nu_vals_int_11 = [0, 1, 2] # to be scaled by 1 / T / M

h_vals_21 = np.array([1.6, 0.8 + 0.6j, -0.5 + 0.7j])
tau_vals_int_21 = [0, 2, 1] # to be scaled by T / N
nu_vals_int_21 = [0, 1, 2] # to be scaled by 1 / T / M

h_vals_12 = np.array([1.0, 0.4 + 0.3j, -0.2 + 0.1j])
tau_vals_int_12 = [0, 1, 2] # to be scaled by T / N
nu_vals_int_12 = [0, 1, 2] # to be scaled by 1 / T / M

h_vals_22 = np.array([1.6, 0.8 + 0.6j, -0.5 + 0.7j])
tau_vals_int_22 = [0, 1, 2] # to be scaled by T / N
nu_vals_int_22 = [0, 1, 2] # to be scaled by 1 / T / M


In [ ]:
received_signal_1 = np.zeros(data_signal_1.shape, dtype='complex')
received_signal_2 = np.zeros(data_signal_1.shape, dtype='complex')

for i in range(len(h_vals_11)):
    delay = tau_vals_int_11[i]
    doppler_shift = nu_vals_int_11[i] / M / T
    n_samples_changed = delay * n_samples_per_T // N
    if delay == 0:
        delayed_s = data_signal_1
    else:
        delayed_s = np.concatenate((np.zeros(n_samples_changed, dtype='complex128'), data_signal_1[:-n_samples_changed]))
    received_signal_1 += delayed_s * h_vals_11[i] * np.exp(1j * 2 * np.pi * doppler_shift * t)

for i in range(len(h_vals_12)):
    delay = tau_vals_int_12[i]
    doppler_shift = nu_vals_int_12[i] / M / T
    n_samples_changed = delay * n_samples_per_T // N
    if delay == 0:
        delayed_s = data_signal_2
    else:
        delayed_s = np.concatenate((np.zeros(n_samples_changed, dtype='complex128'), data_signal_2[:-n_samples_changed]))
    received_signal_1 += delayed_s * h_vals_12[i] * np.exp(1j * 2 * np.pi * doppler_shift * t)

for i in range(len(h_vals_21)):
    delay = tau_vals_int_21[i]
    doppler_shift = nu_vals_int_21[i] / M / T
    n_samples_changed = delay * n_samples_per_T // N
    if delay == 0:
        delayed_s = data_signal_1
    else:
        delayed_s = np.concatenate((np.zeros(n_samples_changed, dtype='complex128'), data_signal_1[:-n_samples_changed]))
    received_signal_2 += delayed_s * h_vals_21[i] * np.exp(1j * 2 * np.pi * doppler_shift * t)

for i in range(len(h_vals_22)):
    delay = tau_vals_int_21[i]
    doppler_shift = nu_vals_int_22[i] / M / T
    n_samples_changed = delay * n_samples_per_T // N
    if delay == 0:
        delayed_s = data_signal_2
    else:
        delayed_s = np.concatenate((np.zeros(n_samples_changed, dtype='complex128'), data_signal_2[:-n_samples_changed]))
    received_signal_2 += delayed_s * h_vals_22[i] * np.exp(1j * 2 * np.pi * doppler_shift * t)

In [ ]:
received_data_1 = np.zeros(data_tx_1.shape, dtype='complex')
for k in range(N):
    for l in range(M):
        received_data_1[k, l] = np.trapz(received_signal_1 * np.conj(alpha_signals[k, l]), t)
np.set_printoptions(linewidth=np.inf)
print(received_data_1)
#For Rx-2
received_data_2 = np.zeros(data_tx_1.shape, dtype='complex')
for k in range(N):
    for l in range(M):
        received_data_2[k, l] = np.trapz(received_signal_2 * np.conj(alpha_signals[k, l]), t)
np.set_printoptions(linewidth=np.inf)
print(received_data_2)

[[ 1.02831266+0.01178386j  0.02323123+0.01685392j  0.01786952+0.0194482j   0.01338188+0.02023668j]
 [-0.04536542-0.02938766j  0.21745195+0.38492377j -0.02228695-0.05844657j -0.00181252-0.06530036j]
 [ 0.02658269-0.01887602j  0.0282009 -0.00120566j -0.08093866-0.187398j    0.00431097+0.01679129j]
 [ 0.01473289-0.00335635j  0.01304596+0.00129792j  0.00994681+0.00394207j  0.00692279+0.00438508j]]
[[ 1.68456047+0.05368462j  0.06486743+0.07329986j  0.04089273+0.08258761j  0.01846996+0.08171565j]
 [-0.06701843-0.05282384j -0.06627026-0.07869608j  0.09067121+0.87964734j -0.01678574-0.13764069j]
 [ 0.07837606-0.104208j   -0.73134227+0.09798726j  0.11143952+0.02626319j  0.06619986+0.07514593j]
 [ 0.05318967-0.011566j    0.0509368 +0.00924358j  0.03940383+0.02500691j  0.02335313+0.03180491j]]


In [ ]:
channel_indices_11 = np.argwhere(np.abs(received_data_1) > 0.1)
print(channel_indices_11)
L = channel_indices_11.shape[0]
estimated_tau_vals_int_11 = channel_indices_11[:,1]
estimated_nu_vals_int_11 = channel_indices_11[:,0]
print(f"Estimated delays: {estimated_tau_vals_int_11}")
print(f"Estimated Doppler shifts: {estimated_nu_vals_int_11}")

channel_indices_21 = np.argwhere(np.abs(received_data_2) > 0.15)
print(channel_indices_21)
L = channel_indices_21.shape[0]
estimated_tau_vals_int_21 = channel_indices_21[:,1]
estimated_nu_vals_int_21 = channel_indices_21[:,0]
print(f"Estimated delays: {estimated_tau_vals_int_21}")
print(f"Estimated Doppler shifts: {estimated_nu_vals_int_21}")

# channel_indices_12 = np.argwhere(np.abs(received_data_1) > 0.15)
# print(channel_indices_12)
# L = channel_indices_12.shape[0]
# estimated_tau_vals_int_12 = channel_indices_12[:,1]
# estimated_nu_vals_int_12 = channel_indices_12[:,0]
# print(f"Estimated delays: {estimated_tau_vals_int_12}")
# print(f"Estimated Doppler shifts: {estimated_nu_vals_int_12}")

# channel_indices_22 = np.argwhere(np.abs(received_data_2) > 0.15)
# print(channel_indices_22)
# L = channel_indices_21.shape[0]
# estimated_tau_vals_int_22 = channel_indices_22[:,1]
# estimated_nu_vals_int_22 = channel_indices_22[:,0]
# print(f"Estimated delays: {estimated_tau_vals_int_22}")
# print(f"Estimated Doppler shifts: {estimated_nu_vals_int_22}")

[[0 0]
 [1 1]
 [2 2]]
Estimated delays: [0 1 2]
Estimated Doppler shifts: [0 1 2]
[[0 0]
 [1 2]
 [2 1]]
Estimated delays: [0 2 1]
Estimated Doppler shifts: [0 1 2]


Correction Factor for each combination

In [ ]:
def correction_factors_matrix(channel_indices,estimated_tau_vals_int,estimated_nu_vals_int,data_signal):

  L = channel_indices.shape[0]
  correction_factors = np.zeros((L, L), dtype='complex')
  for m in range(L):
    k_prime, l_prime = channel_indices[m,:]
    for i in range(L):
        delay = estimated_tau_vals_int[i]
        nu_i = estimated_nu_vals_int[i]
        doppler_shift = nu_i / M / T
        n_samples_changed = delay * n_samples_per_T // N
        if delay == 0:
            s = data_signal
        else:
            s = np.concatenate((np.zeros(n_samples_changed, dtype='complex128'), data_signal[:-n_samples_changed]))
        s = s * np.exp(1j * 2 * np.pi * doppler_shift * t)
        correction_factors[m,i] = np.trapz(s * np.conj(alpha_signals[k_prime, l_prime]), t)
  return(correction_factors)

print(correction_factors_matrix(channel_indices_11,estimated_tau_vals_int_11,estimated_nu_vals_int_11,data_signal_2))

[[ 9.98986476e-01+0.00000000e+00j  5.43921629e-02-3.63449898e-02j  3.33451410e-02-3.33484571e-02j]
 [ 6.75869617e-08-1.01426053e-07j  8.17123756e-01+3.38463768e-01j  4.02364548e-02-1.88797768e-03j]
 [ 9.29829407e-08-1.18112322e-18j -4.37960385e-03-2.77102829e-02j -7.75099329e-09+8.75000009e-01j]]


In [ ]:
Y = []
for i in range(L):
    Y.append(received_data_1[channel_indices_11[i, 0], channel_indices_11[i, 1]])
for i in range(L):
    Y.append(received_data_2[channel_indices_21[i, 0], channel_indices_21[i, 1]])

correction_factors_1 = correction_factors_matrix(channel_indices_11,estimated_tau_vals_int_11,estimated_nu_vals_int_11,data_signal_2)
correction_factors_2 = correction_factors_matrix(channel_indices_21,estimated_tau_vals_int_21,estimated_nu_vals_int_21,data_signal_2)

print(correction_factors_1)
print(correction_factors_2)
correction_factors = np.zeros((2*L, 2*L), dtype='complex')
for i in range(L):
  for j in range(L):
      correction_factors[i,j]=correction_factors_1[i,j]

for i in range(L):
  for j in range(L):
      correction_factors[L+i,L+j]=correction_factors_2[i,j]
print(correction_factors)
# alphas = np.kron(np.eye(2),correction_factors)
# print(alphas)
alphas=correction_factors
h_est = np.linalg.inv(alphas) @ Y
print(h_est)
print(h_vals_12,h_vals_22)



[[ 9.98986476e-01+0.00000000e+00j  5.43921629e-02-3.63449898e-02j  3.33451410e-02-3.33484571e-02j]
 [ 6.75869617e-08-1.01426053e-07j  8.17123756e-01+3.38463768e-01j  4.02364548e-02-1.88797768e-03j]
 [ 9.29829407e-08-1.18112322e-18j -4.37960385e-03-2.77102829e-02j -7.75099329e-09+8.75000009e-01j]]
[[ 9.98986476e-01+0.00000000e+00j  5.98033397e-02-2.47734576e-02j  1.97260658e-02-4.76262721e-02j]
 [ 7.60420714e-08-1.09894870e-07j  6.25399445e-01+6.25399466e-01j  2.71164648e-02+2.97865023e-02j]
 [ 8.45194491e-08-1.61900650e-18j -6.55806803e-03-2.72769638e-02j  6.18718428e-01+6.18718448e-01j]]
[[ 9.98986476e-01+0.00000000e+00j  5.43921629e-02-3.63449898e-02j  3.33451410e-02-3.33484571e-02j  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 6.75869617e-08-1.01426053e-07j  8.17123756e-01+3.38463768e-01j  4.02364548e-02-1.88797768e-03j  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 9.29829407e